# SVAMITVA Feature Extraction — Local Mac Training Pipeline (out.ipynb)

**Objective:** Train the model using the explicitly added `Desktop/DATA` folder on your Apple Silicon Mac (MPS).


---
## CELL 1 — Setup & Environment Configuration (Run Once)
This cell dynamically checks variables, imports required codebase files (`feature_extractor.py`, etc.), and allocates Apple Silicon.

In [1]:
import os
import sys
import time
from pathlib import Path

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

# ── Workspace Directory Checking ──────────────────────────────────────────
NOTEBOOK_DIR = Path.cwd()
if str(NOTEBOOK_DIR) not in sys.path:
    sys.path.insert(0, str(NOTEBOOK_DIR))

DATA_DIR = Path("/Users/aaronr/Desktop/DATA")

CKPT_DIR = NOTEBOOK_DIR / "checkpoints"
CKPT_DIR.mkdir(parents=True, exist_ok=True)
(NOTEBOOK_DIR / "logs").mkdir(exist_ok=True)

# ── Device Allocation (MPS / CPU) ─────────────────────────────────────────
if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
    print("Apple Silicon GPU (MPS) successfully acquired ✅")
else:
    device = torch.device("cpu")
    print("Executing on CPU ⚠️")

print(f"Targeting Dataset: {DATA_DIR} (Exists: {DATA_DIR.exists()})")

# ── Training Configuration ────────────────────────────────────────────────
CONFIG = dict(
    backbone         = "resnet50",
    pretrained       = True,   # Automatically fetch generic image weights locally
    image_size       = 512,
    batch_size       = 4,      # Optimized for Mac Memory
    epochs_per_map   = 20,     # Faster local preview
    learning_rate    = 2e-4,
    weight_decay     = 1e-4,
    num_workers      = 0,
    mixed_precision  = False,  # MPS AutoCast is unstable
    gradient_clip    = 1.0,
    
    # Priority multi-task loss weightings
    building_weight        = 1.0,
    roof_weight            = 0.5,
    road_weight            = 0.8,
    waterbody_weight       = 0.8,
    road_centerline_weight = 0.7,
    waterbody_line_weight  = 0.7,
    waterbody_point_weight = 0.9,
    utility_line_weight    = 0.7,
    utility_poly_weight    = 0.8,
    bridge_weight          = 1.0,
    railway_weight         = 0.9,
)

TARGET_KEYS = [
    "building_mask", "road_mask", "road_centerline_mask",
    "waterbody_mask", "waterbody_line_mask", "waterbody_point_mask",
    "utility_line_mask", "utility_poly_mask",
    "bridge_mask", "railway_mask", "roof_type_mask",
]

print("\nImports and Configuration Complete ✓")

ModuleNotFoundError: No module named 'torch'

---
## CELL 2 — Training Engine Core
Initializes building blocks from the native `.py` codebase: dataset loading, architecture, logic scaling, focal loss algorithms, backwards passes.

In [ ]:
from models.feature_extractor import FeatureExtractorModel
from models.losses import MultiTaskLoss
from training.metrics import MetricTracker
from data.dataset import SvamitvaDataset
from data.augmentation import get_train_transforms

def move_targets(batch):
    return {k: batch[k].to(device) for k in TARGET_KEYS if k in batch}

def build_model(load_from: Path = None):
    m = FeatureExtractorModel(
        backbone=CONFIG["backbone"],
        pretrained=CONFIG["pretrained"],
        num_roof_classes=5,
    )
    if load_from and load_from.exists():
        state = torch.load(load_from, map_location="cpu")
        m.load_state_dict(state["model"], strict=False)
        print(f"Loaded weights from checkpoint: {load_from.name}")
    return m.to(device)


def train_map(map_name: str, resume_from: Path = None):
    map_dir  = DATA_DIR / map_name
    best_out = CKPT_DIR / f"{map_name}_best.pt"
    last_out = CKPT_DIR / f"{map_name}_latest.pt"

    if not map_dir.exists():
        print(f"[SKIPPING/ERROR] Folder NOT FOUND: {map_dir}")
        return best_out if best_out.exists() else None

    print(f"\n{'='*70}")
    print(f"  Training Engine Activating for Region : {map_name}")
    print(f"  Loading prior state weights           : {resume_from.name if resume_from and resume_from.exists() else 'RANDOM / SCRATCH'}")
    print(f"{'='*70}")

    # Model, Engine Setup
    model_w   = build_model(load_from=resume_from)

    # Dataset Setup
    try:
        ds = SvamitvaDataset(
            root_dir   = DATA_DIR,
            image_size = CONFIG["image_size"],
            transform  = get_train_transforms(CONFIG["image_size"]),
            mode       = "train",
        )
        ds.samples = [s for s in ds.samples if s["map_name"] == map_name]
        print(f"  Generated dynamically cached tiles    : {len(ds)}")
    except Exception as e:
        print(f"Failed to load dataset for {map_name}: {e}")
        return None

    loader = DataLoader(ds, batch_size=CONFIG["batch_size"], shuffle=True, num_workers=CONFIG["num_workers"], pin_memory=False)

    # Loss Setup (Includes BinaryFocalLoss)
    loss_fn = MultiTaskLoss(**{k: v for k, v in CONFIG.items() if k.endswith("_weight")}).to(device)
    optimizer = torch.optim.AdamW(model_w.parameters(), lr=CONFIG["learning_rate"], weight_decay=CONFIG["weight_decay"])
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=CONFIG["epochs_per_map"], eta_min=1e-6)

    # Loop
    best_iou  = 0.0
    for epoch in range(1, CONFIG["epochs_per_map"] + 1):
        model_w.train()
        tracker  = MetricTracker()
        run_loss = 0.0
        n_steps  = 0
        t0       = time.time()

        for batch in loader:
            images  = batch["image"].to(device)
            targets = move_targets(batch)
            optimizer.zero_grad(set_to_none=True)
            
            # Forward Pass (No AMP on Mac)
            preds         = model_w(images)
            total_loss, loss_dict = loss_fn(preds, targets)
            
            # NaN Guard & Backward Pass
            if not torch.isfinite(total_loss):
                print(f"      [NaN SKIP] skipping bad gradients in step...")
                continue 

            total_loss.backward()
            if CONFIG["gradient_clip"] > 0:
                nn.utils.clip_grad_norm_(model_w.parameters(), CONFIG["gradient_clip"])
            optimizer.step()

            run_loss += total_loss.item()
            tracker.update(preds, targets)
            n_steps += 1

        scheduler.step()
        m        = tracker.compute()
        avg_loss = run_loss / max(n_steps, 1)
        avg_iou  = m.get("avg_iou", 0.0)

        print((f"  Epoch {epoch:2d}/{CONFIG['epochs_per_map']} | "
               f"loss: {avg_loss:.4f} | iou: {avg_iou:.4f} | "
               f"time: {time.time()-t0:.0f}s"))

        # Checkpointing
        ckpt = {
            "model"    : model_w.state_dict(),
            "epoch"    : epoch,
            "map_name" : map_name,
            "best_iou" : best_iou,
            "metrics"  : m,
        }
        torch.save(ckpt, last_out)
        if avg_iou > best_iou:
            best_iou = avg_iou
            ckpt["best_iou"] = best_iou
            torch.save(ckpt, best_out)
            print(f"    → New Best Checkpoint saved! IoU = {best_iou:.4f}")

    print(f"\n  [MAP COMPLETE] {map_name} finished.  Best IoU={best_iou:.4f}")
    return best_out

print("Training Core Active ✓")

---
## CELL 3 — Executive Training Suite 
Run this block sequentially. It handles progressive multi-map knowledge assimilation (

In [ ]:
# Execute progressively. It handles missing maps gracefully, carrying over checkpoints across regions 
cpt1 = train_map("MAP1", resume_from=None)

print("\n*** SVAMITVA PIPELINE EXHAUSTED ***")